### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7323,2023-11-26,Espanha Acb,08:30,Breogan,Zaragoza,1.48,2.68,155.5,1.86,1.94,-5.5,2.01,1.76,6BfsIlj9,0.675676,0.373134,0.537634,0.515464,0.048810,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,152.15,250.47,0.0,0.0,0.0,0.0,0.407946,0.029773,0.093781,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7324,2023-11-26,Espanha Acb,08:30,Real Madrid,MoraBanc Andorra,1.07,9.10,169.5,1.88,1.90,-16.5,2.04,1.74,p2goH85F,0.934579,0.109890,0.531915,0.526316,0.044470,0.0,0.0,NaN,NaN,104.226,10.739019,0.103036,NaN,NaN,NaN,0.0,0.0,113.46,168.96,0.0,0.0,0.0,0.0,1.116631,0.007483,0.112239,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7325,2023-11-26,Espanha Acb,13:00,Barcelona,Basquet Girona,1.08,8.40,164.5,1.84,1.96,-15.5,2.07,1.72,2wljGSKL,0.925926,0.119048,0.543478,0.510204,0.044974,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,115.62,0.00,0.0,0.0,0.0,0.0,1.091988,0.044659,0.130600,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7326,2023-11-26,Espanha Acb,13:00,Baskonia,Manresa,1.28,3.75,169.5,1.83,1.91,-9.5,2.10,1.68,KWetd1Ur,0.781250,0.266667,0.546448,0.523560,0.047917,0.0,0.0,NaN,NaN,NaN,NaN,NaN,200.056,83.868103,0.419223,0.0,0.0,131.14,204.30,0.0,0.0,0.0,0.0,0.694455,0.030251,0.157135,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7327,2023-11-26,Espanha Acb,14:30,Gran Canaria,Tenerife,1.58,2.40,159.5,1.81,1.97,-4.5,1.99,1.78,tUnfFnzS,0.632911,0.416667,0.552486,0.507614,0.049578,0.0,0.0,NaN,NaN,172.844,71.266089,0.412315,199.962,110.934588,0.554778,0.0,0.0,109.47,368.52,0.0,0.0,0.0,0.0,0.291371,0.059861,0.078776,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7523,2023-11-26,Eua Ncaa,18:00,Jacksonville St.,Incarnate Word,1.33,3.48,138.5,1.91,1.91,-7.5,2.00,1.71,MZvE9Gt2,0.751880,0.287356,0.523560,0.523560,0.039236,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,124.82,219.20,0.0,0.0,0.0,0.0,0.632133,0.000000,0.110545,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7524,2023-11-26,Eua Ncaa,18:00,Rhode Island,Yale,3.07,1.41,140.5,1.91,1.91,4.5,2.05,1.71,EmA2lVAb,0.325733,0.709220,0.523560,0.523560,0.034953,0.0,0.0,NaN,NaN,NaN,NaN,NaN,172.834,156.827241,0.907387,0.0,0.0,83.49,94.25,0.0,0.0,0.0,0.0,0.524017,0.000000,0.127881,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7525,2023-11-26,Eua Ncaa,18:30,Delaware,George Washington,1.80,2.05,150.5,1.80,1.95,-2.5,2.00,1.71,zR4MPH96,0.555556,0.487805,0.555556,0.512821,0.043360,0.0,0.0,NaN,NaN,NaN,NaN,NaN,129.582,48.756943,0.376263,0.0,0.0,92.30,215.82,0.0,0.0,0.0,0.0,0.091832,0.056569,0.110545,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
7526,2023-11-26,Eua Ncaa,20:30,Iowa State,Texas A&M,1.91,1.91,135.5,1.91,1.91,-2.5,2.10,1.63,409QOyPC,0.523560,0.523560,0.523560,0.523560,0.047120,0.0,0.0,NaN,NaN,NaN,NaN,NaN,104.552,26.284638,0.251403,0.0,0.0,0.00,116.13,0.0,0.0,0.0,0.0,0.000000,0.000000,0.178198,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
5,00:30,Eua Nba,Los Angeles Clippers,Dallas Mavericks,235.5,1.89,1.0000,Over
11,21:30,Eua Nba,Brooklyn Nets,Chicago Bulls,214.5,1.94,0.9439,Over
12,21:30,Eua Nba,Cleveland Cavaliers,Toronto Raptors,218.5,1.94,0.8596,Over
68,17:00,Eua Ncaa,Texas,Wyoming,142.5,1.91,0.8943,Over
71,18:00,Eua Ncaa,UCF Knights,Stetson,139.5,1.91,0.8734,Over
75,19:00,Eua Ncaa,West Virginia,Bellarmine,131.5,1.91,0.8830,Over
77,20:00,Eua Ncaa,Florida Intl,Kennesaw State,158.5,1.91,0.8593,Over
